# Some stats

This is a copy of @kapral42's great notebook https://www.kaggle.com/kapral42/covid-19-in-russia-regions made to do some experiments with graphs

# **Import**

In [ ]:
# essential libraries
import json
import random
from urllib.request import urlopen

# storing and anaysis
import numpy as np
import pandas as pd
from scipy import stats as sps
from scipy.interpolate import interp1d

# visualization
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
import folium

from matplotlib import pyplot as plt
from matplotlib.dates import date2num, num2date
from matplotlib import dates as mdates
from matplotlib import ticker
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

# color pallette
cnf = '#393e46' # confirmed - grey
dth = '#ff2e63' # death - red
rec = '#21bf73' # recovered - cyan
act = '#fe9801' # active case - yellow
tst = '#2fb1d2' # tests - blue
ctl = '#e2e42e' # contrloed - yellow

# converter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()   

# hide warnings
import warnings
warnings.filterwarnings('ignore')

# html embedding
from IPython.display import Javascript
from IPython.core.display import display
from IPython.core.display import HTML

# Datasets

* [Russia dataset](https://www.kaggle.com/kapral42/covid19-russia-regions-cases)
* [World dataset](https://www.kaggle.com/kapral42/covid19yandexdataset)

In [ ]:
# !ls ../input/covid19-russia-regions-cases/
# !ls ../input/covid19yandexdataset/
# !ls ../input/estimating-covid-19-rt-in-real-time-russia/

In [ ]:
# Read Russian data
# rus_df = pd.read_csv('../input/covid19-russia-regions-cases/covid19-russia-cases.csv')
rus_df = pd.read_csv('../input/covid19-russia-regions-cases/covid19-russia-cases-scrf.csv')
rus_other_df = pd.read_csv('../input/covid19-russia-regions-cases/covid19-tests-and-other.csv')
rus_info_df = pd.read_csv('../input/covid19-russia-regions-cases/regions-info.csv')
rus_vent_df = pd.read_csv('../input/covid19-russia-regions-cases/regions-ventilators.csv')
# iso_df = pd.read_csv('../input/covid19-russia-regions-cases/isolation_daily.csv')
iso_df = pd.read_csv('../input/covid19yandexdataset/ID.csv')
mos_death_df = pd.read_csv('../input/covid19-russia-regions-cases/moscow_deaths.csv')

# Read world data
world_df = pd.read_csv('../input/covid19yandexdataset/WS.csv')

# Read R-value notebook output
rt_df = pd.read_csv('../input/estimating-covid-19-rt-in-real-time-russia/rt.csv')

In [ ]:
# Data preporation
rus_df['Active'] = rus_df['Confirmed'] - rus_df['Deaths'] - rus_df['Recovered']
rus_df['Death_rate'] = rus_df['Deaths'] / rus_df['Confirmed'] * 100
world_df['Active'] = world_df['Confirmed'] - world_df['Deaths'] - world_df['Recovered']
world_df['Death_rate'] = (world_df['Deaths'] / world_df['Confirmed'] * 100).fillna(0)

# Remove Diamond Princess cases
rus_df = rus_df[rus_df['Region/City'] != 'Diamond Princess']

rt_df.date = pd.to_datetime(rt_df.date)
iso_df.Date = pd.to_datetime(iso_df.Date, dayfirst=True)
world_df.Date = pd.to_datetime(world_df.Date, dayfirst=True)
rus_df['Date'] = pd.to_datetime(rus_df['Date'], dayfirst=True) 
print('Russian data actual date:', rus_df.iloc[-1]['Date'].strftime('%d, %b %Y'))
print('World data actual date:', world_df.sort_values('Date').iloc[-1]['Date'].strftime('%d, %b %Y'))

cases = ['Day-Confirmed', 'Day-Deaths', 'Day-Recovered', 'Confirmed', 'Deaths', 'Recovered', 'Active']

In [ ]:
# Combining with regions info
right = rus_info_df[['Region_ID', 'Population']]
rus_df = rus_df.merge(right, left_on='Region_ID', right_on='Region_ID', how='left')
rus_df['Conf_p100k'] = rus_df.Confirmed / (rus_df.Population / 100000)
rus_df = rus_df.drop('Population', axis=1)

In [ ]:
# Get current situation in regions
rus_latest = rus_df.groupby('Region/City').apply(lambda df: df.loc[df['Date'].idxmax()])
rus_latest = rus_latest.sort_values(by='Confirmed', ascending=False).reset_index(drop=True)
rus_latest = rus_latest[['Region/City'] + cases + ['Death_rate', 'Conf_p100k']]

# Moscow/Russia

In [ ]:
temp = rus_df.copy()
temp = temp.drop(columns=['Region/City-Eng', 'Region_ID','Day-Deaths', 'Day-Recovered', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Death_rate', 'Conf_p100k'])
temp.loc[(temp['Region/City'] != 'Москва'),'Region/City']='Россия'
temp = temp.groupby(['Date','Region/City']).sum().reset_index().sort_values(['Date','Region/City'])
#temp.style.background_gradient(cmap='Reds')

#merged.groupby(["contributor_firstname", "contributor_lastname"]).amount.sum().reset_index().sort_values("amount", ascending=False)


In [ ]:
#temp1 = temp.groupby(['Date', 'Region/City'])[['Day-Confirmed', 'Confirmed']].max().reset_index()
#temp1 = temp1.sort_values(['Confirmed', 'Date'], ascending=False)

px.line(temp, x="Date", y="Day-Confirmed", color='Region/City', title='Daily confirmed cases (Москва/Россия кроме Москвы)', height=600)

# Region wise Data

In [ ]:
rus_latest.style.background_gradient(cmap='Reds')

In [ ]:
temp = rus_df.groupby(['Date', 'Region/City'])['Confirmed'].max().reset_index()
temp = temp.sort_values(['Confirmed', 'Date'], ascending=False)

px.line(temp, x="Date", y="Confirmed", color='Region/City', title='Cases Spread', height=600)

In [ ]:
temp = rus_df.groupby(['Date', 'Region/City'])['Conf_p100k'].max().reset_index()
temp = temp.sort_values(['Conf_p100k', 'Date'], ascending=False)

fig = px.line(temp, x="Date", y="Conf_p100k", color='Region/City', title='Confirmed cases per 100 000 people', height=600)
fig.update_layout(yaxis_title='Confirmed per 100K')

In [ ]:
temp = rus_df.groupby(['Date', 'Region/City'])[['Day-Confirmed', 'Confirmed']].max().reset_index()
temp = temp.sort_values(['Confirmed', 'Date'], ascending=False)

px.line(temp, x="Date", y="Day-Confirmed", color='Region/City', title='Daily confirmed cases spread', height=600)

In [ ]:
top = 30
temp = rus_latest.loc[rus_latest.Death_rate > 0].sort_values('Death_rate', ascending=False).head(top)
temp = temp.sort_values('Death_rate').melt(id_vars="Region/City", value_vars=['Death_rate'])

fig = px.bar(temp, y="Region/City", x="value", color='variable', color_discrete_sequence=[dth, cnf], orientation='h')
fig.update_layout(barmode='group', title=f'The percentage of deaths from all confirmed cases - Top {top}',
                  xaxis_title='Death Rate', yaxis_title='', height=800)
fig.show()

In [ ]:
# Copied function from "Estimating COVID-19 Rt in Real-Time. Russia" notebook
def plot_rt(result, ax, state_name):
    
    ax.set_title(f"{state_name}")
    
    # Colors
    ABOVE =  [1,0,0]
    MIDDLE = [1,1,1]
    BELOW =  [0,0,0]
    cmap = ListedColormap(np.r_[
        np.linspace(BELOW,MIDDLE,25),
        np.linspace(MIDDLE,ABOVE,25)
    ])
    color_mapped = lambda y: np.clip(y, .5, 1.5)-.5
    
    index = result['ML'].index.get_level_values('date')
    values = result['ML'].values
    
    # Plot dots and line
    ax.plot(index, values, c='k', zorder=1, alpha=.25)
    ax.scatter(index,
               values,
               s=40,
               lw=.5,
               c=cmap(color_mapped(values)),
               edgecolors='k', zorder=2)
    
    # Aesthetically, extrapolate credible interval by 1 day either side
    lowfn = interp1d(date2num(index),
                     result['Low_90'].values,
                     bounds_error=False,
                     fill_value='extrapolate')
    
    highfn = interp1d(date2num(index),
                      result['High_90'].values,
                      bounds_error=False,
                      fill_value='extrapolate')
    
    extended = pd.date_range(start=pd.Timestamp('2020-03-21'),
                             end=index[-1]+pd.Timedelta(days=1))
    
    ax.fill_between(extended,
                    lowfn(date2num(extended)),
                    highfn(date2num(extended)),
                    color='k',
                    alpha=.1,
                    lw=0,
                    zorder=3)

    ax.axhline(1.0, c='k', lw=1, label='$R_t=1.0$', alpha=.25);
    
    # Formatting
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    ax.xaxis.set_minor_locator(mdates.DayLocator())
    
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:.1f}"))
    ax.yaxis.tick_right()
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.margins(0)
    ax.grid(which='major', axis='y', c='k', alpha=.1, zorder=-2)
    ax.margins(0)
    ax.set_ylim(0.0, 5.0)
    ax.set_xlim(pd.Timestamp('2020-03-21'), result.index.get_level_values('date')[-1]+pd.Timedelta(days=1))
    fig.set_facecolor('w')


**Estimated COVID-19's  Rt  in Real-Time** 
* [Detailed description](https://www.kaggle.com/fo2rist/estimating-covid-19-rt-in-real-time-russia)

In [ ]:
print('Rt data actuality date:', rt_df.iloc[-1]['date'].strftime('%d, %b %Y'))

In [ ]:
ncols = 4
nrows = int(np.ceil(len(rt_df.groupby('state')) / ncols))

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, nrows*3))

for i, (state_name, result) in enumerate(rt_df.groupby('state')):
    if len(result) < 2:
        print(f"Not enough data to plot: {state_name}")
        continue
    plot_rt(result.set_index('date'), axes.flat[i], state_name)

fig.tight_layout()
fig.set_facecolor('w')

# Russian data

In [ ]:
# Russia data by days
day_cases = ['Day-Confirmed', 'Day-Deaths', 'Day-Recovered']
rus_sum_df = rus_df.loc[rus_df['Region/City'] != 'Diamond Princess']
rus_sum_df = rus_sum_df.groupby('Date').apply(lambda df: df[day_cases].sum())
rus_sum_df['Country/Region'] = 'Russia'
rus_sum_df = rus_sum_df.groupby('Country/Region').apply(lambda df: df[day_cases].cumsum()).reset_index()
rus_sum_df['Country/Region'] = 'Russia'
rus_sum_df = rus_sum_df.rename(columns={'Day-Confirmed': 'Confirmed', 'Day-Deaths': 'Deaths', 'Day-Recovered': 'Recovered'})

rus_sum_df['Day-Confirmed'] = rus_sum_df.groupby('Country/Region')['Confirmed'].diff()
rus_sum_df['Day-Deaths'] = rus_sum_df.groupby('Country/Region')['Deaths'].diff()
rus_sum_df['Day-Recovered'] = rus_sum_df.groupby('Country/Region')['Recovered'].diff()
rus_sum_df.loc[0, ['Day-Confirmed', 'Day-Deaths', 'Day-Recovered']] = rus_sum_df.loc[0, ['Confirmed', 'Deaths', 'Recovered']].values 

rus_sum_df['Active'] = rus_sum_df['Confirmed'] - rus_sum_df['Deaths'] - rus_sum_df['Recovered']
rus_sum_df['Death_rate'] = rus_sum_df['Deaths'] / rus_sum_df['Confirmed'] * 100

# rus_sum_df.tail()

Russia summary table

In [ ]:
cm = sns.light_palette("green", as_cmap=True)
style = (rus_sum_df.drop('Country/Region', axis=1).style
         .bar(subset=['Confirmed', 'Deaths'], color='#67000d')
         .bar(subset=['Recovered'], color='#00441b')
         .background_gradient(cmap='Reds')
         .background_gradient(cmap='Greens', subset=['Day-Recovered', 'Recovered'])    
        )
style

In [ ]:
temp = rus_sum_df.melt(id_vars="Date", value_vars=['Recovered', 'Deaths', 'Active'],
                 var_name='Case', value_name='Count')

fig = px.area(temp, x="Date", y="Count", color='Case',
             title='Russia situation over time', color_discrete_sequence = [rec, dth, act])

temp1 = rus_sum_df[rus_sum_df['Date'] >= pd.Timestamp('2020-03-25')]
fig.add_trace(go.Scatter(x=temp1.Date, y=temp1.Death_rate, name="Death Rate %", yaxis="y2", line_color=dth))
fig.update_layout(yaxis2=dict(overlaying="y", side="left", position=0.26))

fig.show()


## Comparison with other countries
Use "Days from firs case" instead "Date" column

In [ ]:
rus_days_df = rus_sum_df[['Date', 'Country/Region', 'Confirmed', 'Deaths', 'Recovered', 'Death_rate', 'Active', 'Day-Confirmed']]
rus_days_df = rus_days_df.rename(columns={'Country/Region' : 'Country', 'Day-Confirmed' : 'Day_Confirmed'})
first_day = rus_days_df['Date'][0]
rus_days_df['Days'] = rus_days_df.groupby('Date').apply(lambda df: df['Date'] - first_day).reset_index(drop=True)

columns = rus_days_df.columns
# threshold = rus_days_df['Confirmed'].max() + 10000

def world_top_conf(world_df, n):
    top = world_df.groupby('Country').Confirmed.max()
    top = top.sort_values(ascending=False).head(n)
    return top

def country_data(world_df, country):
    country_df = world_df.loc[world_df.Country == country]
#     country_df = country_df.loc[(country_df.Confirmed > 0) & (country_df.Confirmed <= threshold)].reset_index(drop=True)
    country_df = country_df.loc[(country_df.Confirmed > 0)].reset_index(drop=True)

    first_day = country_df.Date[0]
    country_df['Days'] = country_df.groupby('Date').apply(lambda df: df.Date - first_day).reset_index(drop=True)
    country_df = country_df[columns]
    return country_df

w_top_conf = world_top_conf(world_df, 10)

top_list = []
for c in w_top_conf.index.values:
    top_list.append(country_data(world_df, c))


In [ ]:
temp = pd.concat(top_list)

temp['Days'] = temp['Days'].astype('str')
temp1 = temp.groupby(['Country', 'Date']).apply(lambda df: int(df['Days'][0].split(' ')[0])).reset_index()
temp1 = temp1.sort_values(['Date', 'Country']).reset_index(drop=True)
temp = temp.sort_values(['Date', 'Country']).reset_index(drop=True)
temp['Days'] = temp1[0]

fig = px.line(temp, x="Days", y="Confirmed", color='Country',
              title='Number of cases in the most infected countries', height=600)
fig.update_layout(xaxis_title='Days from the first case')

fig.show()

In [ ]:
fig = px.line(temp, x="Days", y="Active", color='Country',
              title='Active cases in the most infected countries', height=600)
fig.update_layout(xaxis_title='Days from the first case')

fig.show()

In [ ]:
fig = px.line(temp, x="Days", y="Death_rate", color='Country',
              title='Death rate in the most infected countries', height=600)
fig.update_layout(xaxis_title='Days from the first case')

fig.show()

In [ ]:
fig = px.line(temp, x="Days", y="Day_Confirmed", color='Country',
              title='New cases per day in the most infected countries', height=600)
fig.update_layout(xaxis_title='Days from the first case')

fig.show()

In [ ]:

rus_other_df.Date = pd.to_datetime(rus_other_df.Date, dayfirst=True)

temp = rus_other_df[rus_other_df['Date'] >= pd.Timestamp('2020-02-16')]

fig = go.Figure()
fig.add_trace(go.Scatter(x=temp.Date, y=temp.Tests,
                         line=dict(color=tst, width=2), mode='lines+markers', name='Tests performed'))
fig.update_layout(title='Number of tests performed (not number of tested people)',
                   xaxis_title='Date')
fig.show()

In [ ]:
rus_tests_df = rus_other_df[rus_other_df['Date'] >= pd.Timestamp('2020-03-02')][['Date', 'Tests']]
rus_tests_df = rus_tests_df[~np.isnan(rus_tests_df.Tests)]
rus_tests_df['Day_tests'] = rus_tests_df.Tests.diff()
rus_tests_df = rus_tests_df[~np.isnan(rus_tests_df.Day_tests) & (rus_tests_df.Day_tests >= 0)]

temp = rus_sum_df[rus_sum_df['Date'] >= pd.Timestamp('2020-03-02')]

fig = go.Figure(data=[
    go.Bar(name='Day-Confirmed', x=temp.Date, y=temp['Day-Confirmed'], marker_color=cnf),
    go.Bar(name='Tests performed', x=rus_tests_df.Date, y=rus_tests_df.Day_tests, marker_color=tst)
])
# Change the bar mode
fig.update_layout(barmode='stack', yaxis_type="log", title='Tests performed and confirmed cases per day in Russia')
fig.show()

In [ ]:
temp = rus_sum_df.melt(id_vars="Date", 
                 value_vars=['Day-Confirmed', 'Day-Deaths', 'Day-Recovered'])

fig = px.bar(temp, x="Date", y="value", color='variable', 
             color_discrete_sequence=[cnf, dth, rec])
fig.update_layout(barmode='group', title='New registered cases in Russia per day', yaxis_title='')
fig.show()

In [ ]:
# Growth factor
def growth_factor(df):
    df['Growth-Factor'] = 1
    for i in range(1, len(df)):
        df.loc[i, 'Growth-Factor'] = df.loc[i, 'Day-Confirmed'] / df.loc[i-1, 'Day-Confirmed']
    
    df['Growth-Factor-Week'] = 1
    for i in range(7, len(df)):
        gf = 0
        for j in range(7):
            gf += df.loc[i-j, 'Day-Confirmed'] / df.loc[i-j-1, 'Day-Confirmed']
        df.loc[i, 'Growth-Factor-Week'] = gf / 7
    return df

rus_sum_df = growth_factor(rus_sum_df)

**Growth Factor**

GF = Daily-Confirmed[i] / Daily-Confirmed[i-1]

GFW = mean(GF[i-7] ... GF[i])

In [ ]:
temp = rus_sum_df[rus_sum_df['Date'] >= pd.Timestamp('2020-03-1')]
temp = temp.melt(id_vars="Date", 
                 value_vars=['Growth-Factor', 'Growth-Factor-Week'])

fig = px.bar(temp, x="Date", y="value", color='variable', 
             color_discrete_sequence=[cnf, rec, dth],
             title='Growth Factor of confirmed cases in Russia')
fig.update_layout(barmode='group')
fig.add_trace(go.Scatter(x=temp['Date'], y=[1]*len(rus_sum_df),
                         line=dict(color=dth, width=4, dash='dash'), name='Inflection Level'))
fig.show()

In [ ]:
def doubled(df):
    df['Doubled-Days'] = 1
    df['Incr-Fact'] = 2
    for i in reversed(range(1, len(df))):
        dd = 0
        incr = 1.0
        for j in range(1, i):
            incr = df.loc[i, 'Confirmed'] / df.loc[i-j, 'Confirmed']
            dd += 1
            if incr >= 2.0:
                break
        df.loc[i, 'Doubled-Days'] = dd
        df.loc[i, 'Incr-Fact'] = incr
    return df

rus_sum_df = doubled(rus_sum_df)

In [ ]:
temp = rus_sum_df[rus_sum_df['Date'] >= pd.Timestamp('2020-03-1')]
temp = temp.melt(id_vars="Date", 
                 value_vars=['Doubled-Days', 'Incr-Fact'])

fig = px.bar(temp, x="Date", y="value", color='variable', 
             color_discrete_sequence=[cnf, dth],
             title='How long did it take for the number of total confirmed cases to double?')
fig.update_layout(barmode='group', yaxis_title='Days / Factor')
fig.show()

*Incr-Fact - increase factor between today and N days before

Simple prediction by using current Doubled-Days count and Increased-Factor. In real-world the variables constantly changing for various reasons.

In [ ]:
# Prediction by current Doubled-Days and Incr-Fact
def predict(df, d_pred, reg_str='Country/Region', name_suff='Prediction'):
    # Mean values for 3 last days
    dd = int(df.iloc[-3]['Doubled-Days'].mean())
    incf = df.iloc[-1]['Incr-Fact'].mean()
    today = df.iloc[-1]['Date']
#     print(today, 'dd', dd, 'incf', incf)

    pred = df.iloc[-dd:][['Date', 'Confirmed']].reset_index(drop=True)
    for i in range(dd, d_pred + dd):
        day = pred.iloc[i - 1]['Date'] + pd.DateOffset(days=1)
        conf = pred.iloc[i - dd]['Confirmed'] * incf
        pred = pred.append(pd.DataFrame([[day, conf]], columns=pred.columns), ignore_index=True)
#         print('    ', day, conf)
    
    pred[reg_str] = df.iloc[0][reg_str] + name_suff
    return pred, dd, incf

d_pred = 14
rus_pred, dd, incf = predict(rus_sum_df, d_pred)

In [ ]:
# temp = pd.concat([rus_sum_df, rus_pred.iloc[dd-1:]]).reset_index(drop=True)
# px.line(temp, x="Date", y="Confirmed", color='Country/Region',
#         title='Prediction for {} days by current Doubled-Days({}) count and Increased-Factor({:.2})'.format(d_pred, dd, incf), height=600)

In [ ]:
# Compare past predictions with current situation 
# d_pred = 28
# rus_pred1, dd, incf = predict(rus_sum_df.loc[rus_sum_df.Date <= pd.Timestamp('2020-04-08')], d_pred, name_suff='Prediction minus a week')

# temp1 = pd.concat([temp, rus_pred1.iloc[dd-1:]]).reset_index(drop=True)
# px.line(temp1, x="Date", y="Confirmed", color='Country/Region',
#         title='Prediction for {} days by current Doubled-Days({}) count and Increased-Factor({:.2})'.format(d_pred, dd, incf), height=600)

In [ ]:
state_name = 'Россия'

result = rt_df.loc[rt_df.state == state_name].set_index('date')
fig, ax = plt.subplots(figsize=(800/72,600/72))
plot_rt(result, ax, state_name)
ax.set_title(f'Real-time $R_t$ for {state_name}')
ax.xaxis.set_major_locator(mdates.WeekdayLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))

In [ ]:
iso_df.head()

In [ ]:
temp = iso_df.groupby('Date')['Index'].mean().reset_index().sort_values('Date')
temp = temp.melt(id_vars="Date", 
                 value_vars=['Index'])

fig = px.bar(temp, x="Date", y="value", color='value', title='Russia Yandex self-isolation index')
fig.update_layout(barmode='group', yaxis_title='Index')

fig.show()

In [ ]:
temp = rus_other_df[rus_other_df['Date'] >= pd.Timestamp('2020-02-16')]
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp.Date, y=temp.Control, name='For all the period', fill='tozeroy', line_color=cnf))
fig.add_trace(go.Scatter(x=temp.Date, y=temp.Control_active, name='At given day', fill='tozeroy', line_color=act))
fig.update_layout(title='People under medical supervision',
                   xaxis_title='Date')
fig.show()

In [ ]:
top = 20
# temp = rus_vent_df.head(top).sort_values('Vent_idx').melt(id_vars="Region_eng", 
#                    value_vars=['Vent_idx'])

# fig = px.bar(temp, y="Region_eng", x="value", color='variable', 
#              color_discrete_sequence=[tst, dth, rec], orientation='h')
# fig.update_layout(barmode='group', title='Ventilators per 100 000 people - Top {} region list'.format(top),
#                   xaxis_title='Index value', yaxis_title='')
# fig.show()

In [ ]:
top = 20
# temp = rus_vent_df.sort_values('ECMO_idx', ascending=False).head(top).sort_values('ECMO_idx')
# temp = temp.melt(id_vars="Region_eng", value_vars=['ECMO_idx'])

# fig = px.bar(temp, y="Region_eng", x="value", color='variable', 
#              color_discrete_sequence=[dth, rec], orientation='h')
# fig.update_layout(barmode='group', title='ECMO per 100 000 people - Top {} region list'.format(top),
#                   xaxis_title='Index value', yaxis_title='')
# fig.show()

# Moscow

Moscow summary table

In [ ]:
moscow_cases = rus_df.loc[rus_df['Region/City'] == 'Москва'].reset_index(drop=True)

cm = sns.light_palette("green", as_cmap=True)
style = (moscow_cases.drop(['Region/City', 'Region/City-Eng', 'Region_ID'], axis=1).style
         .bar(subset=['Confirmed', 'Deaths'], color='#67000d')
         .bar(subset=['Recovered'], color='#00441b')
         .background_gradient(cmap='Reds')
         .background_gradient(cmap='Greens', subset=['Day-Recovered', 'Recovered'])    
        )
style

In [ ]:
temp = moscow_cases.melt(id_vars="Date", 
                 value_vars=['Day-Confirmed', 'Day-Deaths', 'Day-Recovered'])

fig = px.bar(temp, x="Date", y="value", color='variable', 
             color_discrete_sequence=[cnf, dth, rec])
fig.update_layout(barmode='group', title='New registered cases in Moscow per day', yaxis_title='')
fig.show()

In [ ]:
temp = moscow_cases.groupby('Date')['Recovered', 'Deaths', 'Active'].sum().reset_index()
temp = temp.melt(id_vars="Date", value_vars=['Recovered', 'Deaths', 'Active'],
                 var_name='Case', value_name='Count')
temp.head()

fig = px.area(temp, x="Date", y="Count", color='Case',
             title='Moscow cases over time', color_discrete_sequence = [rec, dth, act])

temp1 = moscow_cases[moscow_cases['Date'] >= pd.Timestamp('2020-03-25')]
fig.add_trace(go.Scatter(x=temp1.Date, y=temp1.Death_rate, name="Death Rate %", yaxis="y2", line_color=dth))
fig.update_layout(yaxis2=dict(overlaying="y", side="left", position=0.19))

fig.show()

In [ ]:
# Growth factor
moscow_cases = growth_factor(moscow_cases)

temp = moscow_cases[moscow_cases['Date'] >= pd.Timestamp('2020-03-1')]
temp = temp.melt(id_vars="Date", 
                 value_vars=['Growth-Factor', 'Growth-Factor-Week'])

fig = px.bar(temp, x="Date", y="value", color='variable', 
             color_discrete_sequence=[cnf, rec, dth],
             title='Growth Factor of confirmed cases in Moscow')
fig.update_layout(barmode='group')
fig.add_trace(go.Scatter(x=temp['Date'], y=[1]*len(rus_sum_df),
                         line=dict(color=dth, width=4, dash='dash'), name='Inflection Level'))
fig.show()

In [ ]:
moscow_cases = doubled(moscow_cases)

temp = moscow_cases[moscow_cases['Date'] >= pd.Timestamp('2020-03-1')]
temp = temp.melt(id_vars="Date", 
                 value_vars=['Doubled-Days', 'Incr-Fact'])

fig = px.bar(temp, x="Date", y="value", color='variable', 
             color_discrete_sequence=[cnf, dth],
             title='How long did it take for the number of total confirmed cases to double?')
fig.update_layout(barmode='group', yaxis_title='Days / Factor')
fig.show()

In [ ]:
d_pred = 14
moscow_pred, dd, incf = predict(moscow_cases, d_pred, 'Region/City-Eng')

# temp = pd.concat([moscow_cases, moscow_pred.iloc[dd-1:]]).reset_index(drop=True)
# px.line(temp, x="Date", y="Confirmed", color='Region/City-Eng',
#         title='Prediction for {} days by current Doubled-Days({}) count and Increased-Factor({:.2})'.format(d_pred, dd, incf), height=600)

**Estimated Moscow COVID-19's  Rt  in Real-Time** 
* [Detailed description](https://www.kaggle.com/fo2rist/estimating-covid-19-rt-in-real-time-russia)

In [ ]:
state_name = 'Москва'

result = rt_df.loc[rt_df.state == state_name].set_index('date')
fig, ax = plt.subplots(figsize=(800/72,600/72))
plot_rt(result, ax, state_name)
ax.set_title(f'Real-time $R_t$ for {state_name}')
ax.xaxis.set_major_locator(mdates.WeekdayLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))

[Yandex self-isolation index](https://yandex.ru/company/researches/2020/podomam) -- [Map](https://yandex.ru/maps/covid19/isolation?ll=81.460685%2C55.691496&z=4)

In [ ]:
temp = iso_df[iso_df.City == 'Moscow'][['Date', 'Index']].sort_values('Date')
temp = temp.melt(id_vars="Date", 
                 value_vars=['Index'])

fig = px.bar(temp, x="Date", y="value", color='value', title='Moscow Yandex self-isolation index')
fig.update_layout(barmode='group', yaxis_title='Index')

fig.show()

Additional mortality = (Deaths for current month) - mean(Values of deaths for current month per 5 previous years)

In [ ]:
# mos_death_df.tail()

In [ ]:
# Add may 2020 from another official source https://mosgorzdrav.ru/ru-RU/news/default/card/4122.html
mos_death_df = mos_death_df.append({'Month' : 5, 'Year' : 2020, 'Deaths' : 15713}, ignore_index=True)

In [ ]:
# Moscow deaths March-May
mos_month_deaths = {}
mos_mean_deaths = {}
mos_add_deaths = {}
mos_add_post = {}
mos_with_covid = {}
for i in range(4, 6):
    mos_month_deaths[i] = rus_df.loc[(rus_df['Region/City'] != 'Москва') &
                              (rus_df.Date >= pd.Timestamp(f'2020-{i}-1')) &
                              (rus_df.Date < pd.Timestamp(f'2020-{i+1}-1'))
                              , 'Day-Deaths'].sum()
    
    mos_mean_deaths[i] = mos_death_df.loc[(mos_death_df.Month == i) & (mos_death_df.Year >= 2015) & (mos_death_df.Year <= 2019), 'Deaths'].mean()
    mos_add_deaths[i] = mos_death_df.loc[(mos_death_df.Month == i) & (mos_death_df.Year == 2020), 'Deaths'].values[0] - mos_mean_deaths[i]
    mos_add_post[i] = 0
    mos_with_covid[i] = 0
    if i == 5:
        mos_add_post[i] = 2757 - mos_month_deaths[i]
        mos_with_covid[i] = 2503
    
# print(mos_month_deaths)
# print(mos_mean_deaths)
# print(mos_add_deaths)

temp = pd.DataFrame([mos_month_deaths, mos_mean_deaths, mos_add_deaths]).transpose()
temp.columns = ['Covid', 'Mean', 'Add']
temp['Add_post'] = 0

temp['Unknown'] = temp.Add - temp.Covid
temp.Unknown = temp.Unknown.astype('int')

fig = go.Figure(data=[
    go.Bar(name='COVID-19 as main reason', x=temp.index, y=temp.Covid, text=temp.Covid, textposition='auto', marker_color=cnf),
    go.Bar(name='Other reason', x=temp.index, y=temp.Unknown,  text=temp.Unknown, textposition='auto', marker_color=dth)
])
# Change the bar mode
fig.update_layout(barmode='stack', title='Additional mortality in Moscow', xaxis_title='Month')
fig.show()

# Regions without Moscow and Moscow region

In [ ]:
reg_sum_df = rus_df.loc[(rus_df['Region/City'] != 'Москва') & (rus_df['Region/City'] != 'Московская область')].reset_index(drop=True)

# Regions data by days
reg_sum_df = reg_sum_df.groupby('Date').apply(lambda df: df[day_cases].sum())
reg_sum_df['Country/Region'] = 'Russia regions'
reg_sum_df = reg_sum_df.groupby('Country/Region').apply(lambda df: df[day_cases].cumsum()).reset_index()
reg_sum_df['Country/Region'] = 'Russia regions'
reg_sum_df = reg_sum_df.rename(columns={'Day-Confirmed': 'Confirmed', 'Day-Deaths': 'Deaths', 'Day-Recovered': 'Recovered'})

reg_sum_df['Day-Confirmed'] = reg_sum_df.groupby('Country/Region')['Confirmed'].diff()
reg_sum_df['Day-Deaths'] = reg_sum_df.groupby('Country/Region')['Deaths'].diff()
reg_sum_df['Day-Recovered'] = reg_sum_df.groupby('Country/Region')['Recovered'].diff()
reg_sum_df.loc[0, ['Day-Confirmed', 'Day-Deaths', 'Day-Recovered']] = reg_sum_df.loc[0, ['Confirmed', 'Deaths', 'Recovered']].values 

reg_sum_df['Active'] = reg_sum_df['Confirmed'] - reg_sum_df['Deaths'] - reg_sum_df['Recovered']
reg_sum_df['Death_rate'] = reg_sum_df['Deaths'] / reg_sum_df['Confirmed'] * 100

region_name = 'Russia province regions'

# reg_sum_df.tail()

In [ ]:

cm = sns.light_palette("green", as_cmap=True)
style = (reg_sum_df.drop(['Country/Region'], axis=1).style
         .bar(subset=['Confirmed', 'Deaths'], color='#67000d')
         .bar(subset=['Recovered'], color='#00441b')
         .background_gradient(cmap='Reds')
         .background_gradient(cmap='Greens', subset=['Day-Recovered', 'Recovered'])    
        )
style

In [ ]:
temp = reg_sum_df.melt(id_vars="Date", 
                 value_vars=['Day-Confirmed', 'Day-Deaths', 'Day-Recovered'])

fig = px.bar(temp, x="Date", y="value", color='variable', 
             color_discrete_sequence=[cnf, dth, rec])
fig.update_layout(barmode='group', title=f'New registered cases in {region_name} per day', yaxis_title='')
fig.show()

In [ ]:
temp = reg_sum_df.groupby('Date')['Recovered', 'Deaths', 'Active'].sum().reset_index()
temp = temp.melt(id_vars="Date", value_vars=['Recovered', 'Deaths', 'Active'],
                 var_name='Case', value_name='Count')
temp.head()

fig = px.area(temp, x="Date", y="Count", color='Case',
             title=f'{region_name} cases over time', color_discrete_sequence = [rec, dth, act])

temp1 = reg_sum_df[reg_sum_df['Date'] >= pd.Timestamp('2020-03-28')]
fig.add_trace(go.Scatter(x=temp1.Date, y=temp1.Death_rate, name="Death Rate %", yaxis="y2", line_color=dth))
fig.update_layout(yaxis2=dict(overlaying="y", side="left", position=0.20))

fig.show()

In [ ]:
# Growth factor
reg_sum_df = growth_factor(reg_sum_df)

temp = reg_sum_df[reg_sum_df['Date'] >= pd.Timestamp('2020-03-1')]
temp = temp.melt(id_vars="Date", 
                 value_vars=['Growth-Factor', 'Growth-Factor-Week'])

fig = px.bar(temp, x="Date", y="value", color='variable', 
             color_discrete_sequence=[cnf, rec, dth],
             title=f'Growth Factor of confirmed cases in {region_name}')
fig.update_layout(barmode='group')
fig.add_trace(go.Scatter(x=temp['Date'], y=[1]*len(rus_sum_df),
                         line=dict(color=dth, width=4, dash='dash'), name='Inflection Level'))
fig.show()

In [ ]:
reg_sum_df = doubled(reg_sum_df)

temp = reg_sum_df[reg_sum_df['Date'] >= pd.Timestamp('2020-03-1')]
temp = temp.melt(id_vars="Date", 
                 value_vars=['Doubled-Days', 'Incr-Fact'])

fig = px.bar(temp, x="Date", y="value", color='variable', 
             color_discrete_sequence=[cnf, dth],
             title='How long did it take for the number of total confirmed cases to double?')
fig.update_layout(barmode='group', yaxis_title='Days / Factor')
fig.show()